# Talks markdown generator for academicpages

Adapted from generator in academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

Note: edit in Excel and save as tsv. Then open in Notepad and save as with utf-8 encoding.

EDIT 8/19/21: Excel doesn't let me save as tsv anymore? works with txt(tab delimited), didn't seem to need notepad but changed filename below

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
talks = pd.read_csv("talks.txt", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description,tags,image,attr
0,Structure or Nurture? The Effects of Team-Buil...,Conference presentation,psych-safety-cscw,ACM Conference on Computer-Supported Cooperati...,2018-11-05,"Jersey City, NJ",https://emhastings.github.io/files/psych-safet...,Presentation of [conference paper](https://emh...,team,NaN,NaN
1,The History and Construction of Elizabethan En...,Talk,costume-final,Knox College Department of Theatre,2016-05-25,"Galesburg, IL",NaN,Presentation of independent study research. F...,costume,![Margo introduces Emily's work at the beginni...,Peter Bailley
2,Knoxcraft: Teaching Introductory Programming w...,Poster,knoxcraft-horizons,Knox College Horizons Celebration of Student R...,2016-04-08,"Galesburg, IL",https://emhastings.github.io/files/knoxcraft-h...,See [article on Knox website](https://www.knox...,knoxcraft,NaN,NaN
3,Knoxcraft: Teaching Introductory Programming w...,Talk,knoxcraft-ssss,Knox College Summer Science Seminar Series,2015-08-18,"Galesburg, IL",https://emhastings.github.io/files/knoxcraft-s...,Check out the [Knoxcraft website](https://knox...,knoxcraft,NaN,NaN
4,Adventures in Parallel Programming,Pecha Kucha,parallel-horizons,Knox College Horizons Celebration of Student R...,2015-04-10,"Galesburg, IL",https://emhastings.github.io/files/parallel-ho...,See [article on Knox website](https://www.knox...,parallel,NaN,NaN
5,Adventures in Parallel Programming,Talk,parallel-ssss,Knox College Summer Science Seminar Series,2014-08-18,"Galesburg, IL",https://emhastings.github.io/files/parallel-ss...,Winner of the Best Student Seminar Award for 2...,parallel,NaN,NaN
6,The History and Construction of Elizabethan En...,Poster,costume-horizons,Knox College Horizons Celebration of Student R...,2014-02-11,"Galesburg, IL",https://emhastings.github.io/files/costume-hor...,See [article on Knox website](https://www.knox...,costume,![Emily stands by the gown and poster.](https:...,Gay Lynn Hastings
7,Dragonfly Interconnect Topology,Poster,dragonfly-horizons,Knox College Horizons Celebration of Student R...,2014-02-11,"Galesburg, IL",https://emhastings.github.io/files/dragonfly-h...,See [article on Knox website](https://www.knox...,dragonfly,![Emily and Anda stand by their poster.](https...,Gay Lynn Hastings
8,Dragonfly Interconnect Topology,Talk,dragonfly-ssss,Knox College Summer Science Seminar Series,2013-08-08,"Galesburg, IL",https://emhastings.github.io/files/dragonfly-s...,NaN,dragonfly,NaN,NaN
9,The History and Construction of Elizabethan En...,Talk,costume-presentation,Knox College Department of Theatre,2013-05-30,"Galesburg, IL",NaN,Presentation of independent study research. F...,costume,![Emily stands by a mannequin wearing the gown...,Peter Bailley


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [4]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
        
    md += 'excerpt: "' 
    
    if len(str(item.description)) > 3:
        md += item.description + "  \n"
        
    if len(str(item.talk_url)) > 3:
        md += "[Download](" + item.talk_url + ")" 
        
    #close excerpt
    md += '"\n'    
    
    if len(str(item.tags)) > 3:
        md += "tags: [" + html_escape(item.tags) + "]"
           
    md += "\n---\n"
    
    #start of main text
    
    md += "\n" + item.type + "  \n" + item.venue + "  \n"     
    
    if len(str(item.location)) > 3:
        md += html_escape(item.location) + "\n"
    else:
        md += "Virtual\n"
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
    if len(str(item.image)) > 3:
        md += "\n" + html_escape(item.image) + "\n"
        if len(str(item.attr)) > 3:
            md += "\n_Photo by " + html_escape(item.attr) + "._\n"
        else:
            md += "\n_Photo by Emily Hastings._\n"
        
    if len(str(item.talk_url)) > 3:
        if len(str(item.type)) > 3 :
            if item.type == 'Poster':
                md += "\n[Download poster here](" + item.talk_url + ")\n" 
            else: 
                md += "\n[Download slides here](" + item.talk_url + ")\n" 
        else:
            md += "\n[Download here](" + item.talk_url + ")\n" 
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)